In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Load datasets
df_vaccinations = pd.read_csv("us_state_vaccinations.csv")

# Function to calculate total vaccinations by location
def calculate_total_vaccinations():
    # Ensure 'location' column exists and convert total_vaccinations to numeric
    if 'location' in df_vaccinations.columns:
        df_vaccinations['total_vaccinations'] = pd.to_numeric(df_vaccinations['total_vaccinations'], errors='coerce')
        total_vaccinations_by_location = df_vaccinations.groupby('location')['total_vaccinations'].sum()
        return total_vaccinations_by_location
    else:
        print("Error: 'location' column not found in vaccinations DataFrame.")
        return None

# Function to calculate totals for additional columns
def calculate_total_columns():
    # List of columns to calculate totals for
    columns_to_sum = ['total_distributed', 'people_vaccinated', 'people_fully_vaccinated_per_hundred',
                      'total_vaccinations_per_hundred', 'people_fully_vaccinated', 'people_vaccinated_per_hundred',
                      'distributed_per_hundred', 'daily_vaccinations_raw', 'daily_vaccinations',
                      'daily_vaccinations_per_million', 'share_doses_used', 'total_boosters',
                      'total_boosters_per_hundred']
    
    # Convert selected columns to numeric and calculate totals
    for col in columns_to_sum:
        if col in df_vaccinations.columns:
            df_vaccinations[col] = pd.to_numeric(df_vaccinations[col], errors='coerce')
    
    # Group by location and sum the selected columns
    total_columns_by_location = df_vaccinations.groupby('location')[columns_to_sum].sum()
    return total_columns_by_location

# Create dropdown widget for attribute selection
attribute_dropdown = widgets.SelectMultiple(
    options=df_vaccinations.columns,
    value=['total_vaccinations'],  # Default value
    description='Attributes:'
)

# Create dropdown widget for state selection
state_dropdown = widgets.SelectMultiple(
    options=df_vaccinations['location'].unique(),
    value=['Alabama', 'Alaska'],  # Default value
    description='States:'
)

# Define interactive function to update plots based on dropdown selection
@interact(attributes=attribute_dropdown, states=state_dropdown)
def update_plots(attributes, states):
    # Ensure attributes is always a list
    attributes = list(attributes)
    
    # Filter DataFrame based on selected states
    filtered_df = df_vaccinations[df_vaccinations['location'].isin(states)]
    
    # Calculate totals for selected columns
    total_additional_columns = calculate_total_columns()
    
    # Combine the results into a single DataFrame if total_vaccinations is not None
    if total_additional_columns is not None:
        # Visualize totals for selected columns
        plt.figure(figsize=(14, 8))  # Increase figure size
        filtered_df.groupby('location')[attributes].sum().plot(kind='bar', stacked=True)
        plt.title('Totals for Selected Columns by Location')
        plt.xlabel('Location')
        plt.ylabel('Total')
        plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
        plt.legend(title='Column', loc='upper left', bbox_to_anchor=(1, 1))  # Adjust legend placement
        plt.tight_layout()
        plt.show()
    else:
        print("Error: Unable to calculate total columns.")


interactive(children=(SelectMultiple(description='Attributes:', index=(2,), options=('date', 'location', 'tota…